In [1]:
!pip install python-docx Pillow


   ---------------------------------------- 0.0/239.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/239.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/239.6 kB 259.2 kB/s eta 0:00:01
   ----- --------------------------------- 30.7/239.6 kB 259.2 kB/s eta 0:00:01
   ----- --------------------------------- 30.7/239.6 kB 259.2 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/239.6 kB 467.6 kB/s eta 0:00:01
   ------------------------ ------------- 153.6/239.6 kB 538.9 kB/s eta 0:00:01
   ----------------------------------- -- 225.3/239.6 kB 724.0 kB/s eta 0:00:01
   -------------------------------------- 239.6/239.6 kB 732.8 kB/s eta 0:00:00


In [3]:
from docx import Document
from PIL import Image
import io


In [4]:

def reduce_image_quality(image_stream):
    # Open the image using PIL
    image = Image.open(image_stream)

    # Reduce image quality by saving it in JPEG format with reduced quality
    reduced_quality_stream = io.BytesIO()
    image.save(reduced_quality_stream, format="JPEG", quality=50)

    return reduced_quality_stream.getvalue()

def process_word_file(input_path, output_path):
    # Load the Word document
    document = Document(input_path)

    # Iterate through the document elements to find InlineShape elements which usually contain images
    for shape in document.inline_shapes:
        # Only proceed if the shape has an image blob (represents the image data)
        if shape._inline.graphic.graphicData.pic.blipFill.blip.blob is not None:
            # Extract image blob data
            image_stream = io.BytesIO(shape._inline.graphic.graphicData.pic.blipFill.blip.blob)

            # Reduce image quality
            reduced_quality_blob = reduce_image_quality(image_stream)

            # Replace the original image blob with the new lower quality blob
            shape._inline.graphic.graphicData.pic.blipFill.blip.blob = reduced_quality_blob

    # Save the modified document
    document.save(output_path)


In [5]:

if __name__ == "__main__":
    input_path = r"D:\Documents\PDFgear\深基坑专项施工方案（最终） conv 1.docx"
    output_path = r"D:\Documents\PDFgear\深基坑专项施工方案.docx"
    process_word_file(input_path, output_path)
